<h3>Amendments Log</h3>
<table style="width:100%">
  <thead>
    <tr>
      <th style="text-align:left">Version</th>
      <th style="text-align:left">Amended By</th>
      <th style="text-align:left">Date</th>
      <th style="text-align:left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1.0</td>
      <td>Gary Manley</td>
      <td>2025-11-30</td>
      <td>Initial Version</td>
    </tr>
  </tbody>
</table>

# Process Bronze

**Objective:** Process the Bronze Layer using a metadata-driven approach.

**Architecture:**
1. **Read Config:** Query `MovieReleases.main.bronze_config`.
2. **Loop & Merge:** Apply SCD2 logic (end-dating old records) to move data from `landing` to `bronze`.


In [ ]:
# 1. SETUP & IMPORTS
import duckdb
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load Utils
sys.path.append(os.getcwd())
try:
    from utils.db_utils import f_process_scd2
except ImportError:
    print("Error: Could not import utils")

# Load Env (Smart Load)
vLocalEnvPath = r"C:\Users\garym\Documents\GitHub\MovieReleases\.env"
if os.path.exists(vLocalEnvPath):
    load_dotenv(dotenv_path=vLocalEnvPath)
else:
    load_dotenv()

vMdToken = os.getenv("MOTHERDUCK_TOKEN")
if not vMdToken: raise RuntimeError("MOTHERDUCK_TOKEN missing")

# Connect
print("Connecting to MotherDuck...")
vCon = duckdb.connect(f"md:?motherduck_token={vMdToken}")

In [ ]:
# PARAMETERS / CONSTANTS
cNotebookName = "process_bronze.ipynb"

## 2. Execution: Run Merges
Iterate through the config table and execute the merge utility for each row.

In [ ]:
# Fetch active configurations
try:
    dfConfig = vCon.sql("SELECT * FROM MovieReleases.main.bronze_config").df()
except:
    print("Config table not found. Run ingest first.")
    dfConfig = pd.DataFrame()

if dfConfig.empty:
    print("No configuration found. Skipping.")
else:
    print(f"Found {len(dfConfig)} tables to process.")

for vIndex, vRow in dfConfig.iterrows():
    print(f"Processing Rule: {vRow['source_table']} -> {vRow['target_table']} ({vRow['merge_pattern']})")
    
    if vRow['merge_pattern'] == 'SCD2':
        # Pass the row as a dictionary to our utility function
        f_process_scd2(vCon, vRow.to_dict())

print("Pipeline Complete.")
vCon.close()